## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
print(city_data_df.head())

   Unnamed: 0        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0           0     Rikitea -23.1203 -134.9692     73.72        75          89   
1           1  Pascagoula  30.3658  -88.5561     73.45       100           1   
2           2     Ushuaia -54.8000  -68.3000     51.46        50          40   
3           3      Barrow  71.2906 -156.7887     30.22        80          90   
4           4      Atuona  -9.8000 -139.0333     77.45        67          13   

   Wind Speed Current Description Country                 Date  
0       10.89     overcast clouds      PF  2021-10-12 12:15:39  
1        0.00           clear sky      US  2021-10-12 12:23:41  
2       11.50    scattered clouds      AR  2021-10-12 12:21:43  
3       20.71     overcast clouds      US  2021-10-12 12:23:41  
4       18.45          few clouds      PF  2021-10-12 12:23:42  


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
4,4,Atuona,-9.8000,-139.0333,77.45,67,13,18.45,few clouds,PF,2021-10-12 12:23:42
8,8,Oistins,13.0667,-59.5333,84.29,74,20,19.57,few clouds,BB,2021-10-12 12:19:29
13,13,Trincomalee,8.5711,81.2335,85.93,63,100,16.62,overcast clouds,LK,2021-10-12 12:24:13
16,16,Mahebourg,-20.4081,57.7000,79.25,73,20,11.50,few clouds,MU,2021-10-12 12:24:14
22,22,Bathsheba,13.2167,-59.5167,84.81,74,20,19.57,few clouds,BB,2021-10-12 12:24:16
30,30,Nabire,-3.3667,135.4833,79.75,85,64,2.15,broken clouds,ID,2021-10-12 12:24:18
35,35,Neyyattinkara,8.4000,77.0833,82.26,86,99,8.79,overcast clouds,IN,2021-10-12 12:24:20
43,43,Camocim,-2.9022,-40.8411,86.52,57,63,13.11,broken clouds,BR,2021-10-12 12:24:22
46,46,Saldanha,-33.0117,17.9442,77.05,72,100,15.93,overcast clouds,ZA,2021-10-12 12:24:23
48,48,Maumere,-8.6199,122.2111,79.21,74,92,6.53,overcast clouds,ID,2021-10-12 12:24:24


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

Unnamed: 0             235
City                   235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
Country                235
Date                   235
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,77.45,few clouds,-9.8000,-139.0333,
8,Oistins,BB,84.29,few clouds,13.0667,-59.5333,
13,Trincomalee,LK,85.93,overcast clouds,8.5711,81.2335,
16,Mahebourg,MU,79.25,few clouds,-20.4081,57.7000,
22,Bathsheba,BB,84.81,few clouds,13.2167,-59.5167,
30,Nabire,ID,79.75,broken clouds,-3.3667,135.4833,
35,Neyyattinkara,IN,82.26,overcast clouds,8.4000,77.0833,
43,Camocim,BR,86.52,broken clouds,-2.9022,-40.8411,
46,Saldanha,ZA,77.05,overcast clouds,-33.0117,17.9442,
48,Maumere,ID,79.21,overcast clouds,-8.6199,122.2111,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]



print(clean_hotel_df.count())

City                   220
Country                220
Max Temp               220
Current Description    220
Lat                    220
Lng                    220
Hotel Name             220
dtype: int64


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search\WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))